# Import TF and MNIST

# Test it on TF 1.9.0

In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Hyperparameters

In [10]:
learning_rate = 1e-3
num_steps = 1000
batch_size = 100
display_step = 100

# Input and Output sizes

In [11]:
# MNIST data input (img shape: 28*28)
num_input = 784
# MNIST total classes (0-9 digits)
num_classes = 10

# Using high level Estimator to simplify code!
# Required TensorFlow 1.3.0 and up

## Define a next_batch FN!

In [12]:
def next_batch_fn(dataset, batch_size=batch_size):
    def _input_fn():
        X = tf.constant(dataset.images)
        Y = tf.constant(dataset.labels, dtype=tf.int32)
        image_batch, label_batch = tf.train.shuffle_batch([X,Y],
                               batch_size=batch_size,
                               capacity=8*batch_size,
                               min_after_dequeue=4*batch_size,
                               enqueue_many=True)
        print("Finished preparing data for shuffled batch.")
        return {'pixels': image_batch} , label_batch
    return _input_fn

## Use estimator.LinearClassifier (softmax)

In [13]:
def estimator_linear_classifier(dataset, log_level=tf.logging.ERROR, num_steps=num_steps):
    print("\n-----Running linear classifier...")
    tf.logging.set_verbosity(log_level)
    pixel_features = tf.contrib.layers.real_valued_column(
        "pixels", dimension=num_input)
    
    classifier = tf.estimator.LinearClassifier(
        feature_columns=[pixel_features],
        n_classes=num_classes,
        optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    )

    print("Start training.")
    classifier.train(
        input_fn=next_batch_fn(dataset.train),
        steps=num_steps)

    print("Finish training.")

    # Evaluate accuracy.
    test_result = classifier.evaluate(
        input_fn=next_batch_fn(dataset.test),
        steps=10)
    print('Accuracy: {0:f}'.format(test_result['accuracy']))


## Evaluate

In [14]:
# estimator_linear_classifier(mnist)
mnist_without_onehot = input_data.read_data_sets("/tmp/data/")
estimator_linear_classifier(mnist_without_onehot)
# estimator_linear_classifier(mnist_without_onehot, tf.logging.INFO, 500)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz

-----Running linear classifier...
Start training.
Finished preparing data for shuffled batch.
Finish training.
Finished preparing data for shuffled batch.
Accuracy: 0.892000


## DNN for MNIST using estimator

In [15]:
def estimator_dnn_classifier(
    dataset, log_level=tf.logging.ERROR, num_steps=num_steps):
    print("\n-----Running linear classifier...")
    tf.logging.set_verbosity(log_level)

    pixel_features = tf.contrib.layers.real_valued_column(
        "pixels", dimension=num_input)
    
    classifier = tf.estimator.DNNClassifier(
        feature_columns=[pixel_features],
        n_classes=10,
        hidden_units=[512,512,256,256,256,128], # notice how simple it is!
        optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),
        activation_fn=tf.nn.relu,
#         dropout=0.5,
    )
    classifier.train(
        input_fn=next_batch_fn(dataset.train),
        steps=num_steps)
    print("Finished training the DNN model")

    print("\n---Evaluating DNN classifier accuracy...")
    result = classifier.evaluate(
        input_fn=next_batch_fn(dataset.test),
        steps=10)
    print('DNN Classifier Accuracy: {0:f}'.format(result['accuracy']))

## Evaluate

In [16]:
estimator_dnn_classifier(mnist_without_onehot, tf.logging.INFO)
# estimator_dnn_classifier(mnist_without_onehot, tf.logging.INFO, 2000)


-----Running linear classifier...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_evaluation_master': '', '_device_fn': None, '_tf_random_seed': None, '_task_type': 'worker', '_save_summary_steps': 100, '_service': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000012D52A90>, '_model_dir': 'C:\\Users\\dgong\\AppData\\Local\\Temp\\tmpx56new9z', '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_session_config': None, '_is_chief': True, '_train_distribute': None}
Finished preparing data for shuffled batch.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tens